In [ ]:
! pip install torch torchvision
! pip install -U git+https://github.com/luca-medeiros/lang-segment-anything.git

In [7]:
import pandas as pd
import numpy as np
from io import BytesIO
import requests
from PIL import Image, ImageDraw
from lang_sam import LangSAM

In [9]:
# read the data containing the time perriods
data = pd.read_csv('data_time.csv')

In [10]:
data.columns

Index(['Unnamed: 0', 'Image URL', 'Artist', 'Title', 'Date', 'Details',
       'Source', 'Description', 'Similar Prints', 'Description_length',
       'Nouns', 'Year', 'Period'],
      dtype='object')

In [11]:
data = data.dropna(subset=['Image URL'])
data['Image URL'] = data['Image URL'].apply(lambda x: x.replace('scaled', 'images'))
def get_image(url):
    response = requests.get(url)
    return Image.open(BytesIO(response.content))

In [14]:
text_prompts = ['Emperor', 'Shogun', 'Government officials', 'Voting', 'Constitution',
              'Courtroom', 'Legal document', 'Contracts', 'Patent', 'Judge', 'Lawyer','Police', 'Prison',
              'School', 'School uniform', 'Textbooks', 'Scientific instrument',
              'Steamship', 'Telegraph poles', 'Brick house',
              'Factory', 'Steam-powered machinery', 'Worker', 'Railway', 'Train', 'Train station',
              'Soldier', 'Gun', 'Military uniform', 'Warship',
              'Kimono', 'Western style clothing', 'Gloves']

In [15]:
# create the datafram to store the results
result = pd.DataFrame(columns=['Year', 'Period' 'Image URL']+text_prompts)
result.columns

Index(['Year', 'PeriodImage URL', 'Emperor', 'Shogun', 'Government officials',
       'Voting', 'Constitution', 'Courtroom', 'Legal document', 'Contracts',
       'Patent', 'Judge', 'Lawyer', 'Police', 'Prison', 'School',
       'School uniform', 'Textbooks', 'Scientific instrument', 'Steamship',
       'Telegraph poles', 'Brick house', 'Factory', 'Steam-powered machinery',
       'Worker', 'Railway', 'Train', 'Train station', 'Soldier', 'Gun',
       'Military uniform', 'Warship', 'Kimono', 'Western style clothing',
       'Gloves'],
      dtype='object')

In [16]:
model = LangSAM()
for i in range(1):
    # get the year, period, and image url
    result.loc[i, 'Year'] = data['Year'].iloc[i]
    result.loc[i, 'Period'] = data['Period'].iloc[i]
    result.loc[i, 'Image URL'] = data['Image URL'].iloc[i]

    # get the image
    url = data['Image URL'].iloc[i]
    image = get_image(url)
    image = image.convert('RGB')

    # predict the text prompts
    for prompt in text_prompts:
        masks, boxes, phrases, logits = model.predict(image, prompt)
        result.loc[i, prompt] = boxes.shape[0]

d:\APPS\Anaconda\envs\cv\lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:3484.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


final text_encoder_type: bert-base-uncased


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Model loaded from C:\Users\Ke\.cache\huggingface\hub\models--ShilongLiu--GroundingDINO\snapshots\a94c9b567a2a374598f05c584e96798a170c56fb\groundingdino_swinb_cogcoor.pth 
 => _IncompatibleKeys(missing_keys=[], unexpected_keys=['label_enc.weight'])


d:\APPS\Anaconda\envs\cv\lib\site-packages\transformers\modeling_utils.py:874: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
d:\APPS\Anaconda\envs\cv\lib\site-packages\torch\utils\checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


KeyboardInterrupt: 

In [ ]:
result.head()

In [ ]:
# save the results
result.to_csv('limited_category_detection.csv', index=False)